In [ ]:
import requests
from datetime import datetime, date, timedelta
from io import StringIO

import boto3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sagemaker
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.serializers import CSVSerializer
from sklearn.metrics import mean_squared_error

from data import passwords

<img src="data/image/design.png" width=900 height=900 />

## Background

**We will build a forecast model to predict the sales of energy drinks for a retail store located in Iowa City,Iowa. We have 1 year historical data, dated back from 2021-06-01 to 2022-07-04. We will use 2021-06-01 to 2022-06-19 as training date and then predict the demand for the dates between 2022-06-20 to 2022-07-04.**

**We use suggested radius of 1.76 km to search for the nearby events to this store. There are 23 venues close to this store within 1.76km radius. By running the Beam and category importance model, we learned that there are six event categories having statistical correlation to the demand which are:**
- Sports
- Public holidays
- School holidays
- Expos
- Obervances
- Severe weather
- Concerts
- Performing arts

## Severe weather events with Demand Impact Patterns and Polygons

* Severe Weather is one of the most impactful event categories in the world. Warnings or alerts of severe weather may lead to disruption and can have a huge influence on demand.
* Severe weather events impact demand before and after an event. PredictHQ’s Demand Impact Pattern accurately captures the leading, lagging and coincident effects of a severe weather event on demand. 
* Easy to use severe weather event features in a forecast through feature API

<img src="data/image/swdip.png" width=900 height=900 />

In [ ]:
category_important_results = [
    "public_holidays",
    "sports",
    "school_holidays",
    "expos",
    "observances",
    "sw",
    "concerts",
    "performing_arts",
]

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
FEATURES_API_URL = "https://api.predicthq.com/v1/features"
ACCESS_TOKEN = passwords.ACCESS_TOKEN
SEVERE_WEATHER_FEATURES = {
    "phq_impact_severe_weather_air_quality_retail",
    "phq_impact_severe_weather_blizzard_retail",
    "phq_impact_severe_weather_cold_wave_retail",
    "phq_impact_severe_weather_cold_wave_snow_retail",
    "phq_impact_severe_weather_cold_wave_storm_retail",
    "phq_impact_severe_weather_dust_retail",
    "phq_impact_severe_weather_dust_storm_retail",
    "phq_impact_severe_weather_flood_retail",
    "phq_impact_severe_weather_heat_wave_retail",
    "phq_impact_severe_weather_hurricane_retail",
    "phq_impact_severe_weather_thunderstorm_retail",
    "phq_impact_severe_weather_tornado_retail",
    "phq_impact_severe_weather_tropical_storm_retail",
}
SCHOOL_HOLIDAYS_FEATURE = "phq_attendance_school_holidays"


def get_features_api_severe_weather_events(lat, lon, start, end, rank_threshold=30):
    start = datetime.strptime(start, DATE_FORMAT).date()
    end = datetime.strptime(end, DATE_FORMAT).date()

    print("Querying Features API...")
    result = []
    for gte, lte in get_date_groups(start, end):
        print(f"{gte} -> {lte}")
        request_data = {
            "location": {"geo": {"lat": lat, "lon": lon, "radius": "1m"}},
            "active": {"gte": gte, "lte": lte},
        }
        for feature in SEVERE_WEATHER_FEATURES:
            request_data[feature] = {
                "stats": ["max"],
                "phq_rank": {"gte": rank_threshold},
            }

        try:
            response = requests.post(
                f"{FEATURES_API_URL}",
                headers={"Authorization": f"Bearer {ACCESS_TOKEN}"},
                json=request_data,
            ).json()
        except Exception as e:
            print(e)
            return {}, f"{e}"

        for day in response["results"]:
            features = {"date": day["date"]}
            features.update(
                {f: day[f]["stats"]["max"] for f in SEVERE_WEATHER_FEATURES}
            )
            result.append(features)
    return result, None


def get_date_groups(start, end):
    """
    Features API allows range up to 90 days, so we have to do several requests
    """

    def _split_dates(s, e):
        capacity = timedelta(days=90)
        interval = 1 + int((e - s) / capacity)
        for i in range(interval):
            yield s + capacity * i
        yield e

    dates = list(_split_dates(start, end))
    for i, (d1, d2) in enumerate(zip(dates, dates[1:])):
        if d2 != dates[-1]:
            d2 -= timedelta(days=1)
        yield d1.strftime(DATE_FORMAT), d2.strftime(DATE_FORMAT)


res = get_features_api_severe_weather_events(
    41.6576029, -91.53717840355998, "2021-06-01", "2022-07-04", 60
)
df_severe_weather_features = pd.DataFrame(res[0])
columns_constant = [
    col
    for col in df_severe_weather_features.sum()[1:].to_dict().keys()
    if df_severe_weather_features[col].sum() == 0
]
df_severe_weather_features.drop(columns=columns_constant, inplace=True)
df_severe_weather_features.head(2)

In [ ]:
ATTENDED_FEATURES = [
    "phq_attendance_community",
    "phq_attendance_concerts",
    "phq_attendance_conferences",
    "phq_attendance_expos",
    "phq_attendance_festivals",
    "phq_attendance_performing_arts",
    "phq_attendance_sports",
    "phq_attendance_school_holidays",
]
HOLIDAY_FEATURES = [
    "phq_rank_observances",
    "phq_rank_public_holidays",
]


def get_features_api_data(lat, lon, start, end, radius, rank_threshold=30):
    start = datetime.strptime(start, DATE_FORMAT).date()
    end = datetime.strptime(end, DATE_FORMAT).date()

    print("Querying Features API...")
    result = []
    for gte, lte in get_date_groups(start, end):
        print(f"{gte} -> {lte}")
        request_data = {
            "location": {"geo": {"lat": lat, "lon": lon, "radius": f"{radius}m"}},
            "active": {"gte": gte, "lte": lte},
        }
        for feature in ATTENDED_FEATURES:
            request_data[feature] = {
                "stats": ["sum"],
                "phq_rank": {"gte": rank_threshold},
            }

        for feature in HOLIDAY_FEATURES:
            request_data[feature] = True

        try:
            response = requests.post(
                f"{FEATURES_API_URL}",
                headers={"Authorization": f"Bearer {ACCESS_TOKEN}"},
                json=request_data,
            ).json()
        except Exception as e:
            return {}, f"{e}"

        for day in response["results"]:
            # print(day)
            features = {"date": day["date"]}
            features.update({f: day[f]["stats"]["sum"] for f in ATTENDED_FEATURES})
            features.update(
                {f: sum(day[f]["rank_levels"].values()) for f in HOLIDAY_FEATURES}
            )
            result.append(features)
    return result, None


res = get_features_api_data(
    41.6576029, -91.53717840355998, "2021-06-01", "2022-07-30", 1760, 30
)
df_attended_holidays = pd.DataFrame(res[0])
columns_constant = [
    col
    for col in df_attended_holidays.columns[1:]
    if col.replace("phq_attendance_", "").replace("phq_rank_", "")
    not in category_important_results
]
df_attended_holidays.drop(columns=columns_constant, inplace=True)
df_attended_holidays.head(2)

In [ ]:
if "sw" in category_important_results:
    df_event_features = df_attended_holidays.merge(
        df_severe_weather_features, on="date", how="left"
    )
else:
    df_event_features = df_attended_holidays

### Load demand and event feature through csv files

In [ ]:
# Load demand dataset and event features
df_demand = pd.read_csv("data/demand.csv")
df_demand["date"] = pd.to_datetime(df_demand["date"])
df_event_features["date"] = pd.to_datetime(df_event_features["date"])

### Combine event features with time trend features
#### (3 layers: day of week, week of year, month of year)

In [ ]:
# Convert date to time relevant feature
df_event_features[["day_of_week", "week_of_year", "month_of_year"]] = (
    df_event_features["date"]
    .map(lambda x: [x.day_of_week, x.weekofyear, x.month])
    .to_list()
)
df = df_demand.merge(df_event_features, how="left", on="date")

### Build a forecast using XGBoost model based on the above features

In [ ]:
split_date_test = "2022-06-20"
feature_columns = df.columns[2:].tolist()
demand_column = "demand"

X_train = df[df["date"] < split_date_test][
    [demand_column] + feature_columns
]  # for SageMaker XGBoost training input, the first column is target
X_test = df[df["date"] >= split_date_test][feature_columns]
y_test = df[df["date"] >= split_date_test][demand_column]

In [ ]:
feature_columns

In [ ]:
# data management
bucket = sagemaker.Session().default_bucket()
prefix = "demo-xgb-demand-forecasting"
output_path = f"s3://{bucket}/{prefix}/demand_forecast/output"
train_path = f"s3://{bucket}/{prefix}/demand_forecast/train.csv"

# upload training data
X_train.to_csv(train_path, index=False, header=False)

In [ ]:
# initialize hyperparameters
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#parameters-for-tree-booster
hyperparameters = {
    "learning_rate": "0.1",
    "max_depth": "6",
    "objective": "reg:squarederror",  # default
    "seed": "42",
    "num_round": "100",  # equivalent to n_estimators
    "verbosity": "1",
}

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
region = "us-west-2"
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

# construct a SageMaker estimator that calls the xgboost-container
xgb_model = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size=1,  # 1 GB
    output_path=output_path,
)

# define the data type and paths to the training and validation datasets
train_input = TrainingInput(train_path, content_type="text/csv")

# execute the XGBoost training job
xgb_model.fit({"train": train_input})

In [ ]:
xgb_predictor = xgb_model.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium", serializer=CSVSerializer()
)

In [ ]:
response = xgb_predictor.predict(X_test.values).decode("utf-8")
y_prediction = np.array([float(x) for x in response.split()])
y_prediction

### Forecast the next two weeks' demand starting from 2022-06-20

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df[df["date"] < split_date_test]["date"],
        y=df[df["date"] < split_date_test][demand_column],
        name="y_training",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=y_prediction,
        name="y_prediction",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=df[df["date"] >= split_date_test][demand_column],
        name="y_truth",
        mode="lines+markers",
    )
)

# Display the figure
fig.show()

<img src="data/image/sports.png" width=900 height=900 />

### Compare the forecast with event features againest that without event features

In [ ]:
df_withoutevents = df[
    ["date", "demand", "day_of_week", "week_of_year", "month_of_year"]
]
df_withoutevents.head(2)

In [ ]:
feature_columns_withoutevents = df_withoutevents.columns[2:].tolist()
X_train_withoutevents = df_withoutevents[df_withoutevents["date"] < split_date_test][
    [demand_column] + feature_columns_withoutevents
]
X_test_withoutevents = df_withoutevents[df_withoutevents["date"] >= split_date_test][
    feature_columns_withoutevents
]
y_test_withoutevents = df_withoutevents[df_withoutevents["date"] >= split_date_test][
    demand_column
]

In [ ]:
output_path_withoutevents = (
    f"s3://{bucket}/{prefix}/demand_forecast/output_withoutevents"
)
train_path_withoutevents = (
    f"s3://{bucket}/{prefix}/demand_forecast/train_withoutevents.csv"
)

# upload training data
X_train_withoutevents.to_csv(train_path_withoutevents, index=False, header=False)

In [ ]:
# initialize hyperparameters
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#parameters-for-tree-booster
hyperparameters = {
    "learning_rate": "0.1",
    "max_depth": "6",
    "objective": "reg:squarederror",  # default
    "seed": "42",
    "num_round": "100",  # equivalent to n_estimators
    "verbosity": "1",
}

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
region = "us-west-2"
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

# construct a SageMaker estimator that calls the xgboost-container
xgb_model_withoutevents = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size=1,  # 1 GB
    output_path=output_path_withoutevents,
)

# define the data type and paths to the training and validation datasets
train_input = TrainingInput(train_path_withoutevents, content_type="text/csv")

# execute the XGBoost training job
xgb_model_withoutevents.fit({"train": train_input})

In [ ]:
xgb_predictor_withoutevents = xgb_model_withoutevents.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium", serializer=CSVSerializer()
)

In [ ]:
response = xgb_predictor_withoutevents.predict(X_test_withoutevents.values).decode(
    "utf-8"
)
y_prediction_withoutevents = np.array([float(x) for x in response.split()])
y_prediction_withoutevents

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] < split_date_test]["date"],
        y=df_withoutevents[df_withoutevents["date"] < split_date_test][demand_column],
        name="y_training",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] >= split_date_test]["date"],
        y=y_prediction_withoutevents,
        name="y_prediction_no_events",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_withoutevents[df_withoutevents["date"] >= split_date_test]["date"],
        y=df_withoutevents[df_withoutevents["date"] >= split_date_test][demand_column],
        name="y_truth",
        mode="lines+markers",
    )
)
fig.add_trace(
    go.Scatter(
        x=df[df["date"] >= split_date_test]["date"],
        y=y_prediction,
        name="y_prediction_withevents",
        mode="lines+markers",
    )
)
# Display the figure
fig.show()

### Model comparison based on Mean Absolute Error (MAE) and Root Mean Square Error (RMSE)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

MAE_model_withevents = mean_absolute_error(y_test, y_prediction)
MAE_model_no_events = mean_absolute_error(
    y_test_withoutevents, y_prediction_withoutevents
)
MAE_Model_improvement = (
    (MAE_model_no_events - MAE_model_withevents) / MAE_model_no_events * 100
)

RMSE_model_withevents = mean_squared_error(y_test, y_prediction, squared=False)
RMSE_model_no_events = mean_squared_error(
    y_test_withoutevents, y_prediction_withoutevents, squared=False
)
RMSE_Model_improvement = (
    (RMSE_model_no_events - RMSE_model_withevents) / RMSE_model_no_events * 100
)

print(f"MAE for forecasting with events is {MAE_model_withevents:.2f}")
print(f"MAE for forecasting without events is {MAE_model_no_events:.2f}")
print(
    f"MAE improvement of having event features in a forecast model is {MAE_Model_improvement:.2f}%"
)
print(" ")
print(f"RMSE for forecasting with events is {RMSE_model_withevents:.2f}")
print(f"RMSE for forecasting without events is {RMSE_model_no_events:.2f}")
print(
    f"RMSE improvement of having event features in a forecast model is {RMSE_Model_improvement:.2f}%"
)